# Base dos Dados - Auxilio Emergencial x Inflação
## Importando bases utilizadas

In [1]:
#Importando módulos
import pandas as pd
import os
import basedosdados as bd
import statsmodels.api as sm
from patsy import dmatrices

In [2]:
#Importando base de População
pop = bd.read_table('br_ibge_populacao', 'municipios', 
                   billing_project_id = 'base-dos-dados-2')

#Formatando nome das cidades
cidades = []
for i in range(len(pop)):
    if pop['id_municipio'][i] ==2304400:
        cidade = 'Fortaleza (CE)'
    elif pop['id_municipio'][i] ==2611606:
        cidade = 'Recife (PE)'
    elif pop['id_municipio'][i] == 1200401:
        cidade = 'Rio Branco (AC)'
    elif pop['id_municipio'][i] == 2111300:
        cidade = 'São Luís (MA)'
    elif pop['id_municipio'][i] == 2800308:
        cidade = 'Aracaju (SE)'
    elif pop['id_municipio'][i] == 5002704:
        cidade = 'Campo Grande (MS)'
    elif pop['id_municipio'][i] == 5208707:
        cidade = 'Goiânia (GO)'
    elif pop['id_municipio'][i] == 5300108:
        cidade = 'Brasília (DF)'
    elif pop['id_municipio'][i] == 1501402:
        cidade = 'Belém (PA)'
    elif pop['id_municipio'][i] == 3106200:
        cidade = 'Belo Horizonte (MG)'
    elif pop['id_municipio'][i] == 3205309:
        cidade = 'Grande Vitória (ES)'
    elif pop['id_municipio'][i] == 3304557:
        cidade = 'Rio de Janeiro (RJ)'
    elif pop['id_municipio'][i] == 3550308:
        cidade = 'São Paulo (SP)'
    elif pop['id_municipio'][i] == 4106902:
        cidade = 'Curitiba (PR)'
    elif pop['id_municipio'][i] == 4314902:
        cidade = 'Porto Alegre (RS)'  
    elif pop['id_municipio'][i] == 2927408:
        cidade = 'Salvador (BA)'  
    else:
        cidade = 'None'
    cidades.append(cidade)

#Selecionando apenas cidades desejadas (que tem informação específica sobre sua inflação)
pop['Cidades'] = cidades
pop_capitais = pop[(pop['Cidades'] != 'None') & (pop['ano'] == 2020)].drop(columns = {'ano'})

pop_capitais

Downloading: 100%|██████████| 163248/163248 [00:07<00:00, 22722.55rows/s]


,id_municipio,populacao,Cidades
1925,1200401,413418.0,Rio Branco (AC)
4964,1501402,1499641.0,Belém (PA)
18363,2111300,1108975.0,São Luís (MA)
27280,2304400,2686612.0,Fortaleza (CE)
46312,2611606,1653461.0,Recife (PE)
51020,2800308,664908.0,Aracaju (SE)
63239,2927408,2886698.0,Salvador (BA)
67632,3106200,2521564.0,Belo Horizonte (MG)
93037,3205309,365855.0,Grande Vitória (ES)
95027,3304557,6747815.0,Rio de Janeiro (RJ)


In [5]:
#Importando base do Auxílio emergencial
auxilio = pd.read_csv(r'C:\Users\femdi\OneDrive\Documentos\Python\PyCharm\Base dos Dados - Datathon\auxilio_emergencial.csv')

auxilio

,f0_,sigla_uf,id_municipio
0,8132400.0,RR,1400506.0
1,9843600.0,AP,1600253.0
2,101476200.0,AC,1200203.0
3,23227200.0,AC,1200336.0
4,15650400.0,RO,1100080.0
...,...,...,...
5566,16094400.0,SP,3547601.0
5567,1656000.0,SP,3549300.0
5568,10089600.0,SP,3553955.0
5569,4212000.0,SP,3554904.0


In [6]:
#Importando base do IPCA
ipca = pd.read_excel(
    os.path.join(r'C:\Users\femdi\OneDrive\Documentos\Python\PyCharm\Base dos Dados - Datathon',
                 'IPCA2.xlsx'),
    header = 3, 
    index_col=0,
     engine='openpyxl')[0:16]

#arrumando colunas
ipca1 = ipca.reset_index().rename(columns = {'index':'Cidades'}).drop(['Unnamed: 10','Unnamed: 1'], axis = 1)

ipca1

,Cidades,Índice geral,11.Alimentação no domicílio,1101002.Arroz,1101051.Feijão - mulatinho,1101052.Feijão - preto,1101053.Feijão - macáçar (fradinho),1101073.Feijão - carioca (rajado),1107.Carnes
0,Rio Branco (AC),6.12,24.14,67.96,-,-,-,26.58,37.99
1,São Luís (MA),5.71,21.01,61.07,44.02,-,-,25.14,24.54
2,Aracaju (SE),4.14,18.66,66.71,-,-,-,17.79,32.53
3,Campo Grande (MS),6.85,20.05,75.31,-,-,-,-4.45,25.38
4,Goiânia (GO),4.33,18.49,79.09,-,-,-,12.59,17.78
5,Brasília (DF),3.40,14.18,72.14,-,48.51,-,7.52,14.83
6,Belém (PA),4.63,19.61,71.28,-,43.91,-,15.63,18.52
7,Fortaleza (CE),5.74,19.90,82.74,-,-,81.44,-,15.66
8,Recife (PE),5.66,16.83,70.12,32.85,-,28.93,23.05,21.02
9,Salvador (BA),4.31,17.58,74.24,25.87,-,-,12.52,26.60


In [8]:
#Importação base do PIB por município
pib = bd.read_table('br_ibge_pib', 'municipios', 
                   billing_project_id = 'base-dos-dados-2')

pib_2018 = pib[pib['ano'] == 2018]
pib_2018

Downloading: 100%|██████████| 94616/94616 [00:10<00:00, 9459.31rows/s]


,id_municipio,ano,PIB,impostos_liquidos,VA,VA_agropecuaria,VA_industria,VA_servicos,VA_ADESPSS
89046,1100015,2018,499305982,28261302,471044680,165892171,26369018,123512405,155271086
89047,1100023,2018,2463773260,252260341,2211512920,163120731,339667233,1034183951,674541006
89048,1100031,2018,140502269,5651465,134850804,62337414,5178976,25170517,42163897
89049,1100049,2018,2180454016,225198416,1955255600,212492509,222558359,971609245,548595488
89050,1100056,2018,472178834,68863359,403315475,61973278,22818149,208728017,109796030
...,...,...,...,...,...,...,...,...,...
94611,5222005,2018,440221991,30529632,409692358,160485052,28773779,157535566,62897961
94612,5222054,2018,322359328,20921903,301437425,124138070,38649684,97341117,41308553
94613,5222203,2018,103408136,5330540,98077596,29890633,12474430,26585442,29127091
94614,5222302,2018,163782378,6041708,157740669,90989060,12341378,27241232,27168999


## Cruzamentos

In [9]:
#Adicionando valor do auxílio a base de população
pop_capitais1 = pop_capitais.merge(auxilio, how = 'left', on = 'id_municipio')

#Adicionando PIB por municipios
pop_capitais2 = pop_capitais1.merge(pib_2018[['id_municipio', 'PIB']], how = 'left', on = 'id_municipio')

pop_capitais2

,id_municipio,populacao,Cidades,f0_,sigla_uf,PIB
0,1200401,413418.0,Rio Branco (AC),3.725964e+08,AC,8940823180
1,1501402,1499641.0,Belém (PA),1.570426e+09,PA,31484849366
2,2111300,1108975.0,São Luís (MA),1.119480e+09,MA,33605800980
3,2304400,2686612.0,Fortaleza (CE),2.599524e+09,CE,67024088204
4,2611606,1653461.0,Recife (PE),1.444315e+09,PE,52401487302
5,2800308,664908.0,Aracaju (SE),5.395734e+08,SE,17276300787
6,2927408,2886698.0,Salvador (BA),2.598547e+09,BA,63526092487
7,3106200,2521564.0,Belo Horizonte (MG),1.613330e+09,MG,91957091802
8,3205309,365855.0,Grande Vitória (ES),2.260500e+08,ES,26380111447
9,3304557,6747815.0,Rio de Janeiro (RJ),5.239304e+09,RJ,364052058232


In [10]:
#Cruzando com base do ipca
ipca_final = ipca1.merge(pop_capitais2, how = 'left', on = 'Cidades')

#Criando Colunas de Auxilio/população e Auxilio/PIB
ipca_final['Soma auxílio/População da  cidade'] = [ipca_final['f0_'][i]/ipca_final['populacao'][i] for i in range(len(ipca_final))]
ipca_final['Soma auxílio/PIB da cidade'] = [ipca_final['f0_'][i]/ipca_final['PIB'][i] for i in range(len(ipca_final))]

#Renomeando Colunas
ipca_final.columns = ['Cidades', 'Índice geral - IPCA', 'IPCA - Alimentação no domicílio',
       'IPCA - Arroz', 'IPCA - Feijão - mulatinho', 'IPCA - Feijão - preto',
       'IPCA - Feijão - macáçar (fradinho)',
       'IPCA - Feijão - carioca (rajado)', 'IPCA - Carnes', 'id_municipio',
       'populacao', 'valor_auxílio', 'sigla_uf', 'PIB',
       'Soma auxílio/População da  cidade', 'Soma auxílio/PIB da cidade']

ipca_final

,Cidades,Índice geral - IPCA,IPCA - Alimentação no domicílio,IPCA - Arroz,IPCA - Feijão - mulatinho,IPCA - Feijão - preto,IPCA - Feijão - macáçar (fradinho),IPCA - Feijão - carioca (rajado),IPCA - Carnes,id_municipio,populacao,valor_auxílio,sigla_uf,PIB,Soma auxílio/População da cidade,Soma auxílio/PIB da cidade
0,Rio Branco (AC),6.12,24.14,67.96,-,-,-,26.58,37.99,1200401,413418.0,3.725964e+08,AC,8940823180,901.258291,0.041674
1,São Luís (MA),5.71,21.01,61.07,44.02,-,-,25.14,24.54,2111300,1108975.0,1.119480e+09,MA,33605800980,1009.472711,0.033312
2,Aracaju (SE),4.14,18.66,66.71,-,-,-,17.79,32.53,2800308,664908.0,5.395734e+08,SE,17276300787,811.500839,0.031232
3,Campo Grande (MS),6.85,20.05,75.31,-,-,-,-4.45,25.38,5002704,906092.0,6.799380e+08,MS,29177495779,750.407243,0.023304
4,Goiânia (GO),4.33,18.49,79.09,-,-,-,12.59,17.78,5208707,1536097.0,1.082806e+09,GO,49364257877,704.907047,0.021935
5,Brasília (DF),3.40,14.18,72.14,-,48.51,-,7.52,14.83,5300108,3055149.0,1.944686e+09,DF,254817204692,636.527318,0.007632
6,Belém (PA),4.63,19.61,71.28,-,43.91,-,15.63,18.52,1501402,1499641.0,1.570426e+09,PA,31484849366,1047.201430,0.049879
7,Fortaleza (CE),5.74,19.90,82.74,-,-,81.44,-,15.66,2304400,2686612.0,2.599524e+09,CE,67024088204,967.584452,0.038785
8,Recife (PE),5.66,16.83,70.12,32.85,-,28.93,23.05,21.02,2611606,1653461.0,1.444315e+09,PE,52401487302,873.509929,0.027562
9,Salvador (BA),4.31,17.58,74.24,25.87,-,-,12.52,26.60,2927408,2886698.0,2.598547e+09,BA,63526092487,900.179790,0.040905


In [11]:
#Exportando para CSV para mexer no Data Studio
ipca_final.to_csv(r'C:\Users\femdi\OneDrive\Documentos\Python\PyCharm\Base dos Dados - Datathon\base_datathon.csv',
             index = False)

## Regressões

In [12]:
#Selecionando as colunas que serão usadas e tirando os espaços dos nomes
ipca_regressao = ipca_final[['Índice geral - IPCA', 'Soma auxílio/PIB da cidade', 'IPCA - Carnes']]
ipca_regressao = ipca_regressao.rename(columns = {'Índice geral - IPCA':'Indice_geral_IPCA',
                                                  'Soma auxílio/PIB da cidade':'Soma_auxílio_PIB_da_cidade',
                                                  'IPCA - Carnes':'IPCA_Carnes'})
ipca_regressao

,Indice_geral_IPCA,Soma_auxílio_PIB_da_cidade,IPCA_Carnes
0,6.12,0.041674,37.99
1,5.71,0.033312,24.54
2,4.14,0.031232,32.53
3,6.85,0.023304,25.38
4,4.33,0.021935,17.78
5,3.40,0.007632,14.83
6,4.63,0.049879,18.52
7,5.74,0.038785,15.66
8,5.66,0.027562,21.02
9,4.31,0.040905,26.60


In [13]:
'''Regressão IPCA geral'''

#Determinando X e Y 
Y_geral,X_geral = dmatrices('Indice_geral_IPCA ~ Soma_auxílio_PIB_da_cidade',
                data = ipca_regressao, return_type='dataframe')

#Estimando o modelo
model_geral = sm.OLS(Y_geral,X_geral)
fit_geral = model_geral.fit()

#Mostrando Resultados
fit_geral.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      Indice_geral_IPCA   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     2.660
Date:                Thu, 11 Mar 2021   Prob (F-statistic):              0.125
Time:                        21:42:51   Log-Likelihood:                -19.720
No. Observations:                  16   AIC:                             43.44
Df Residuals:                      14   BIC:                             44.99
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      4.1732      0.474      8.812      0.000       3.157       5.189
Soma_auxílio_PIB_da_cidade    27.7957     17.044      1.631      0.125      -8.760      64.352
==============================================================================
Omnibus:                        2.043   Durbin-Watson:                   2.381
Prob(Omnibus):                  0.360   Jarque-Bera (JB):                1.198
Skew:                           0.666   Prob(JB):                        0.549
Kurtosis:                       2.855   Cond. No.                         76.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
'''Regressão IPCA carnes'''

#Determinando X e Y
Y_carne,X_carne = dmatrices('IPCA_Carnes ~ Soma_auxílio_PIB_da_cidade',
                data = ipca_regressao, return_type='dataframe')

#Estimando o modelo
model_carne = sm.OLS(Y_carne,X_carne)
fit_carne = model_carne.fit()

#Mostrando Resultados
fit_carne.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            IPCA_Carnes   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     5.530
Date:                Thu, 11 Mar 2021   Prob (F-statistic):             0.0339
Time:                        21:42:58   Log-Likelihood:                -51.800
No. Observations:                  16   AIC:                             107.6
Df Residuals:                      14   BIC:                             109.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     13.4049      3.517      3.812      0.002       5.862      20.948
Soma_auxílio_PIB_da_cidade   297.6342    126.571      2.352      0.034      26.167     569.101
==============================================================================
Omnibus:                        0.160   Durbin-Watson:                   1.240
Prob(Omnibus):                  0.923   Jarque-Bera (JB):                0.039
Skew:                          -0.037   Prob(JB):                        0.981
Kurtosis:                       2.769   Cond. No.                         76.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""